**Download and Extract from Internet**

In [24]:
!wget https://ftp.gnu.org/gnu/make/make-4.4.tar.gz

--2024-09-14 18:54:24--  https://ftp.gnu.org/gnu/make/make-4.4.tar.gz
Resolving ftp.gnu.org (ftp.gnu.org)... 209.51.188.20, 2001:470:142:3::b
Connecting to ftp.gnu.org (ftp.gnu.org)|209.51.188.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2307891 (2.2M) [application/x-gzip]
Saving to: ‘make-4.4.tar.gz.1’

make-4.4.tar.gz.1   100%[===================>]   2.20M  4.44MB/s    in 0.5s    

2024-09-14 18:54:25 (4.44 MB/s) - ‘make-4.4.tar.gz.1’ saved [2307891/2307891]



In [25]:
!tar -xf make-4.4.tar.gz

**Mount Google Drive and copy files**

In [26]:
from google.colab import drive
drive.mount('mount')

Drive already mounted at mount; to attempt to forcibly remount, call drive.mount("mount", force_remount=True).


In [27]:
!cp mount/MyDrive/kaggle.json .
!ls

elasticsearch-7.0.0			 kaggle.json  make-4.4.tar.gz	 mount
elasticsearch-7.0.0-linux-x86_64.tar.gz  make-4.4     make-4.4.tar.gz.1  sample_data


In [28]:
!rm kaggle.json
!ls
!gdown --id 1p30EBcwWuxH-wrSO7ujzXjl4uVo1cujU
!ls

elasticsearch-7.0.0			 make-4.4	  make-4.4.tar.gz.1  sample_data
elasticsearch-7.0.0-linux-x86_64.tar.gz  make-4.4.tar.gz  mount
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1p30EBcwWuxH-wrSO7ujzXjl4uVo1cujU
To: /content/kaggle.json
100% 70.0/70.0 [00:00<00:00, 300kB/s]
elasticsearch-7.0.0			 kaggle.json  make-4.4.tar.gz	 mount
elasticsearch-7.0.0-linux-x86_64.tar.gz  make-4.4     make-4.4.tar.gz.1  sample_data


**Authenticate with Google APIs**

In [29]:
from google.colab import auth
auth.authenticate_user()

**R Magics**

In [30]:
%load_ext rpy2.ipython


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [31]:
%%R

x <- 42
print(x)

[1] 42


**Elastic Search Magics**

In [32]:
# install
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.0.0

In [33]:
# start server
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.0.0/bin/elasticsearch'],
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                 )

In [34]:
# wait a bit then test
!curl -X GET "localhost:9200/"

{
  "name" : "0cee78d5fe3e",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "qQC3bXKGS-uF9pdaTGgbxw",
  "version" : {
    "number" : "7.0.0",
    "build_flavor" : "default",
    "build_type" : "tar",
    "build_hash" : "b7e28a7",
    "build_date" : "2019-04-05T22:55:32.697037Z",
    "build_snapshot" : false,
    "lucene_version" : "8.0.0",
    "minimum_wire_compatibility_version" : "6.7.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


In [35]:
from requests import models, Session, Request
from urllib.parse import urljoin
from IPython.core.magic import register_cell_magic
import re

# to display json response
def render(r):
  text = r.text
  if text[0] in "[{":  # really json
    return """
    <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
    <script>
    renderjson.set_show_to_level(1)
    document.body.appendChild(renderjson(%s))
    new ResizeObserver(google.colab.output.resizeIframeToContent).observe(document.body)
    </script>
    """ % text
  else:    # other status text
    return "<pre>%s</pre>" % text
models.Response._repr_html_ = render

In [36]:
# %%es magic
@register_cell_magic
def es(line=None, cell=""):
  cell = re.sub(r'(?m)^\s*#.*\n?','', cell) # remove comment
  line1 = (cell + '\n').find('\n')
  method, path = cell[:line1].split(None, 1)
  body = cell[line1:].strip()
  args = {}
  if body:
    args['data'] = (body + '\n').encode()  # in case _bulk
    args['headers'] = {'Content-Type': 'application/json'}

  rsp = Session().send(
          Request(method, urljoin('http://localhost:9200', path), **args)
            .prepare())
  return rsp

In [37]:
%%es
GET /

<Response [200]>

**Python and Javascript Magics**

In [38]:
%%html

<script>

  var foo = "hello world 1!";
  document.body.append(document.createTextNode(foo))

</script>

In [39]:
from IPython.display import Javascript
Javascript('var foo = "hello world 2!!"; document.body.append(document.createTextNode(foo))')

<IPython.core.display.Javascript object>

In [40]:
from google.colab.output import eval_js
eval_js('41 + 1')

42

In [41]:
from IPython.display import JSON
import IPython.display as idisplay
from google.colab import output

def concat(a, b):
  return JSON({'result': '%s %s' % (a, b)})

output.register_callback('notebook.concat', concat)

In [42]:
idisplay.Javascript('''
(async function() {

  const result = await google.colab.kernel.invokeFunction(
    'notebook.concat',
    ['hello', 'world 4!!!!'],
    {});

  const text = result.data['application/json'].result;

  document.querySelector("#output-area").appendChild(document.createTextNode(text))

})();
''')

<IPython.core.display.Javascript object>

**Photo**

In [43]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode

In [44]:
VIDEO_HTML = """
<video autoplay
 width=800 height=600></video>
<script>
var video = document.querySelector('video')
navigator.mediaDevices.getUserMedia({ video: true })
  .then(stream=> video.srcObject = stream)

var data = new Promise(resolve=>{
  video.onclick = ()=>{
    var canvas = document.createElement('canvas')
    var [w,h] = [video.offsetWidth, video.offsetHeight]
    canvas.width = w
    canvas.height = h
    canvas.getContext('2d')
          .drawImage(video, 0, 0, w, h)
    video.srcObject.getVideoTracks()[0].stop()
    video.replaceWith(canvas)
    resolve(canvas.toDataURL('image/jpeg', %f))
  }
})
</script>
"""

In [45]:
def take_photo(filename='photo.jpg', quality=0.8):
  display(HTML(VIDEO_HTML % quality))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return len(binary)

In [46]:
take_photo()

KeyboardInterrupt: 